In [675]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import matplotlib.colors as colors

* ```loadFromTSP(filename)```
* ```scatterplotOfClusters(data, centroids, labels)```
* ```plotWithinSS(avgWithinSS, **kwargs)```
* ```plotMultipleRuns(data, k, bins=20, nbrRuns=200, **kwargs)```

### Exercise 1

The list of points appears after the keyword NODE_COORD_SECTION. Your first task is to write a function that inputs the name of a TSP file and returns a numpy array, of shape (nbrPoints, 2), of its points.

att48.tsp

In [45]:
import numpy as np
def loadFromTSP(filename):
    # """ Inputs the TSP file named by filename, and returns a (nbrPoints,2)
    # numpy array of the data points."""    
    filename = input("Enter TSP file: ")
    inputfile = open(filename, "r")
    
    print("Opening File", filename, "for reading\n" )
    line = inputfile.readline()
    df = pd.read_csv(inputfile, sep ='\t', header=None)
    
    #print('Nbr of stuff:', data_array.size)
    data_array = df[5:53]
    
    print('Nbr of data points:', data_array.size)
    shape = data_array.shape
    print(shape)
    

    
   
    print('\nFirst five data points: '.format(len(data_array[:5])))
    df.head(5)
    
    print("\nCompleted reading the file \n", filename)
    print(loadFromTSP)
    loadFromTSP('att48.tsp')
    type(filename)


In [677]:


import csv
import numpy as np

with open('att48.tsp', 'r') as f:
    data = list(csv.reader(f, delimiter=";"))

data = np.array(data)
print(data)

[['NAME : att48']
 ['COMMENT : 48 capitals of the US (Padberg/Rinaldi)']
 ['TYPE : TSP']
 ['DIMENSION : 48']
 ['EDGE_WEIGHT_TYPE : ATT']
 ['NODE_COORD_SECTION']
 ['1 6734 1453']
 ['2 2233 10']
 ['3 5530 1424']
 ['4 401 841']
 ['5 3082 1644']
 ['6 7608 4458']
 ['7 7573 3716']
 ['8 7265 1268']
 ['9 6898 1885']
 ['10 1112 2049']
 ['11 5468 2606']
 ['12 5989 2873']
 ['13 4706 2674']
 ['14 4612 2035']
 ['15 6347 2683']
 ['16 6107 669']
 ['17 7611 5184']
 ['18 7462 3590']
 ['19 7732 4723']
 ['20 5900 3561']
 ['21 4483 3369']
 ['22 6101 1110']
 ['23 5199 2182']
 ['24 1633 2809']
 ['25 4307 2322']
 ['26 675 1006']
 ['27 7555 4819']
 ['28 7541 3981']
 ['29 3177 756']
 ['30 7352 4506']
 ['31 7545 2801']
 ['32 3245 3305']
 ['33 6426 3173']
 ['34 4608 1198']
 ['35 23 2216']
 ['36 7248 3779']
 ['37 7762 4595']
 ['38 7392 2244']
 ['39 3484 2829']
 ['40 6271 2135']
 ['41 4985 140']
 ['42 1916 1569']
 ['43 7280 4899']
 ['44 7509 3239']
 ['45 10 2676']
 ['46 6807 2993']
 ['47 5185 3258']
 ['48 3023 194

Each data point, depicted as a disk, is assigned its own cluster, indicated by color. Also, each cluster's centroid is depicted as a square of the same color. The second part of this assignment is to write a function that takes a data set (i.e., a numpy array of points) and the (centroids,labels) tuple that results from running k-means on this data set, and produces scatterplots like those above above, where each cluster gets a distinct color, and each cluster's centroid is also depicted.